In [11]:
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import numpy as np
import pickle
import torch
import astropy.units as u
import seaborn as sns
import pandas as pd

from typing import Tuple, Optional, Union
from itertools import cycle
from astropy.io import fits, ascii
from astropy.coordinates import SkyCoord, Angle, concatenate
from sklearn.model_selection import train_test_split
from neural_net import normalize_arr, get_model, CustomLoss, resume
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as IMBPipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold, LeaveOneOut
from sklearn.metrics import confusion_matrix, precision_recall_curve, f1_score
from sklearn.impute import KNNImputer
from matplotlib.cm import ScalarMappable
from sklearn.preprocessing import StandardScaler

from preprocessing.preprocessing import ab_mag_to_flux

import matplotlib as mpl
plt.rc('text', usetex=True)

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'cmr10'  # Computer Modern Roman
mpl.rcParams['font.size'] = 12  # Adjust the font size as needed

PATH_TO_CLEAN_DATA = '/Users/adamboesky/Research/ay98/clean_data'

# Load YSE and BTS data

In [12]:
bts_df = pd.read_csv('/Users/adamboesky/Research/ay98/clean_data/bts_sne.csv')
bts_df

,ZTFID,IAUID,RA,Dec,peakt,peakfilt,peakmag,peakabs,duration,rise,fade,type,redshift,b,A_V
0,ZTF17aaaeqms,AT2023jkf,01:19:13.43,+31:46:26.1,2102.97,r,17.5839,-,18.001,12.004,5.997,-,-,-30.726791,0.205
1,ZTF17aaajowi,-,11:47:44.37,+14:34:10.4,1362.73,r,18.9963,-,>5.611,2.651,>2.96,-,-,70.564727,0.127
2,ZTF17aaazdba,AT2019azh,08:13:16.95,+22:38:53.9,561.73,g,15.2769,-19.76,66.42,24.318,42.102,TDE,0.022,27.562336,0.122
3,ZTF17aabtvsy,SN2022yei,10:35:32.09,+37:38:59.0,1870.99,r,18.0303,-19.41,>34.229,>6.01,28.219,SN Ia,0.06922,59.641962,0.053
4,ZTF17aabwkio,AT2019sio,07:39:53.05,-02:58:50.6,765.01,g,18.1673,-,5.615,2.657,2.958,-,-,9.367699,0.306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7613,ZTF24aadryxf,AT2024bdy,08:11:39.37,-06:12:46.5,2341.84,g,18.9243,-,>18.07,>2.05,>16.02,-,-,14.717259,0.154
7614,ZTF24aaecshg,AT2024asp,10:27:12.30,+31:57:30.4,2341.85,g,18.9854,-,>12.96,>0,>12.96,-,-,58.406619,0.048
7615,ZTF24aaecxwe,AT2024bho,11:18:48.75,+78:04:57.2,2341.90,r,18.8290,-,>10.704,>0.12,10.584,-,-,37.961778,0.074
7616,ZTF24aaedncr,SN2005X,12:24:00.71,+07:46:39.2,2352.91,r,18.4054,-,>15.13,>11.06,>4.07,-,-,69.586749,0.069


In [13]:
ras = []
decs = []

for ra, dec in zip(bts_df['RA'], bts_df['Dec']):
    ra_ang = Angle(ra.split(',')[0], unit='hourangle')
    dec_ang = Angle(f'{dec.split(",")[0]} degrees')
    ras.append(ra_ang.degree)
    decs.append(dec_ang.degree)

In [14]:
bts_df['ra_deg'] = ras
bts_df['dec_deg'] = decs
bts_df

,ZTFID,IAUID,RA,Dec,peakt,peakfilt,peakmag,peakabs,duration,rise,fade,type,redshift,b,A_V,ra_deg,dec_deg
0,ZTF17aaaeqms,AT2023jkf,01:19:13.43,+31:46:26.1,2102.97,r,17.5839,-,18.001,12.004,5.997,-,-,-30.726791,0.205,19.805958,31.773917
1,ZTF17aaajowi,-,11:47:44.37,+14:34:10.4,1362.73,r,18.9963,-,>5.611,2.651,>2.96,-,-,70.564727,0.127,176.934875,14.569556
2,ZTF17aaazdba,AT2019azh,08:13:16.95,+22:38:53.9,561.73,g,15.2769,-19.76,66.42,24.318,42.102,TDE,0.022,27.562336,0.122,123.320625,22.648306
3,ZTF17aabtvsy,SN2022yei,10:35:32.09,+37:38:59.0,1870.99,r,18.0303,-19.41,>34.229,>6.01,28.219,SN Ia,0.06922,59.641962,0.053,158.883708,37.649722
4,ZTF17aabwkio,AT2019sio,07:39:53.05,-02:58:50.6,765.01,g,18.1673,-,5.615,2.657,2.958,-,-,9.367699,0.306,114.971042,-2.980722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7613,ZTF24aadryxf,AT2024bdy,08:11:39.37,-06:12:46.5,2341.84,g,18.9243,-,>18.07,>2.05,>16.02,-,-,14.717259,0.154,122.914042,-6.212917
7614,ZTF24aaecshg,AT2024asp,10:27:12.30,+31:57:30.4,2341.85,g,18.9854,-,>12.96,>0,>12.96,-,-,58.406619,0.048,156.801250,31.958444
7615,ZTF24aaecxwe,AT2024bho,11:18:48.75,+78:04:57.2,2341.90,r,18.8290,-,>10.704,>0.12,10.584,-,-,37.961778,0.074,169.703125,78.082556
7616,ZTF24aaedncr,SN2005X,12:24:00.71,+07:46:39.2,2352.91,r,18.4054,-,>15.13,>11.06,>4.07,-,-,69.586749,0.069,186.002958,7.777556


In [21]:
# Drop everything with no redshift
bts_clean_df = bts_df[bts_df['redshift'] != '-']
bts_clean_df = bts_clean_df[~pd.isnull(bts_clean_df['redshift'])]
bts_clean_df.reset_index(inplace=True, drop=True)
bts_clean_df

,ZTFID,IAUID,RA,Dec,peakt,peakfilt,peakmag,peakabs,duration,rise,fade,type,redshift,b,A_V,ra_deg,dec_deg
0,ZTF17aaazdba,AT2019azh,08:13:16.95,+22:38:53.9,561.73,g,15.2769,-19.76,66.42,24.318,42.102,TDE,0.022,27.562336,0.122,123.320625,22.648306
1,ZTF17aabtvsy,SN2022yei,10:35:32.09,+37:38:59.0,1870.99,r,18.0303,-19.41,>34.229,>6.01,28.219,SN Ia,0.06922,59.641962,0.053,158.883708,37.649722
2,ZTF17aacldgo,SN2022zxv,03:09:24.35,-04:53:39.2,1897.75,g,18.7979,-18.91,>8.87,>3.85,>5.02,SN Ia,0.072,-50.332472,0.183,47.351458,-4.894222
3,ZTF17aadlxmv,SN2020adv,08:29:47.59,+33:54:22.8,879.69,g,17.9475,-19.34,25.146,10.951,14.195,SN Ia,0.062,34.174702,0.106,127.448292,33.906333
4,ZTF18aaaibml,SN2020buc,10:08:31.27,+09:14:23.0,909.80,r,18.2606,-17.70,69.73,27.302,42.428,SN II,0.0348,47.637802,0.092,152.130292,9.239722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5337,ZTF24aaczgax,SN2024anc,10:58:15.99,+23:41:37.3,2341.77,g,17.2317,-17.64,19.703,10.28,9.423,SN II,0.02128,64.269422,0.046,164.566625,23.693694
5338,ZTF24aadauao,SN2024alp,01:59:32.67,+41:09:13.2,2340.62,g,17.8560,-19.40,>22.765,7.735,>15.03,SN Ia,0.05859,-19.917448,0.182,29.886125,41.153667
5339,ZTF24aadkwni,AT2024aul,10:21:53.23,+00:17:44.2,2339.81,r,17.5125,-17.44,>21.96,>3.89,>18.07,SN II,0.0215,45.192542,0.134,155.471792,0.295611
5340,ZTF24aadkwol,SN2024apt,10:25:36.87,-02:12:39.2,2354.83,g,15.4292,-19.23,31.42,17.427,13.993,SN Ia,0.01850,44.243280,0.124,156.403625,-2.210889


In [22]:
# Write to csv
bts_clean_df.to_csv('/Users/adamboesky/Research/ay98/clean_data/bts_sne_clean.csv')

# Domain transfer and infer host properties